In [1]:
import random, re, math
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import tensorflow as tf, tensorflow.keras.backend as K
# from kaggle_datasets import KaggleDatasets
print('Tensorflow version ' + tf.__version__)
from sklearn.model_selection import KFold

Tensorflow version 2.4.0


In [2]:
# AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH = "../input/train_tfrecords"
# BATCH_SIZE = 16
IMAGE_SIZE = (512, 512)
CLASSES = ['0', '1', '2', '3', '4']
NUM_CLASSES = len(CLASSES)
EPOCHS = 12

FILENAMES = tf.io.gfile.glob(GCS_PATH + "/*tfrec")
split_ind = int(0.9 * len(FILENAMES))
TRAINING_FILENAMES, VALID_FILENAMES = FILENAMES[:split_ind], FILENAMES[split_ind:]

# TRAINING_FILENAMES, VALID_FILENAMES = train_test_split(FILENAMES, test_size=0.2, random_state=420)

# TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + "../input/test_tfrecords/*tfrec")
print("Train TFRecord Files:", len(TRAINING_FILENAMES))
print("Validation TFRecord Files:", len(VALID_FILENAMES))
# print("Test TFRecord Files:", len(TEST_FILENAMES))

Train TFRecord Files: 14
Validation TFRecord Files: 2


In [3]:
AUG_BATCH = 12

In [4]:
def cutmix(image, label, PROBABILITY = 1.0):
#     print(label.shape)
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMAGE_SIZE[0]
    CLASSES = 104
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
#         if len(label.shape)==1:
#             lab1 = tf.one_hot(label[j],NUM_CLASSES)
#             lab2 = tf.one_hot(label[k],NUM_CLASSES)
#         else:
#             lab1 = label[j,]
#             lab2 = label[k,]
#         lab1 = tf.one_hot(label[j],NUM_CLASSES)
#         lab2 = tf.one_hot(label[k],NUM_CLASSES)
        lab1 = label[j,]
        lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
#     print(labs)
#     label2 = labs #tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label#labs

In [5]:
from readTFRecords import *

In [6]:
train_dataset = get_validation_dataset(TRAINING_FILENAMES, BATCH_SIZE=12)
valid_dataset = get_validation_dataset(VALID_FILENAMES, BATCH_SIZE=12)

In [7]:
for img, label in train_dataset.take(2):
    pass

In [8]:
cutmix(img, label)

InvalidArgumentError: Value for attr 'TI' of float is not in the list of allowed values: uint8, int32, int64
	; NodeDef: {{node OneHot}}; Op<name=OneHot; signature=indices:TI, depth:int32, on_value:T, off_value:T -> output:T; attr=axis:int,default=-1; attr=T:type; attr=TI:type,default=DT_INT64,allowed=[DT_UINT8, DT_INT32, DT_INT64]> [Op:OneHot]

In [ ]:
row = 3; col = 4;
for (img,label) in augmented_element.take(1):
    plt.figure(figsize=(12,int(12*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,]/255.0)
    plt.show()
    break